In [1]:
import pandas as pd

In [2]:
data = '../data/data-sample_data-nyctaxi-trips-2009-json_corrigido.json'

In [3]:
df = pd.read_json(data, lines=True)

In [4]:
df.to_csv('../data/data-sample_data-nyctaxi-trips-2009.csv', index=False)

In [5]:
df_csv = pd.read_csv('../data/data-sample_data-nyctaxi-trips-2009.csv')

In [6]:
df_csv.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,tip_amount,tolls_amount,total_amount
0,CMT,2009-04-21T18:51:11.767205+00:00,2009-04-21T18:57:09.433767+00:00,2,0.80,-74.004114,40.742950,NaN,NaN,-73.994712,40.747950,Cash,5.4,0.0,0.0,0.0,5.4
1,CMT,2009-01-13T07:40:07.639754+00:00,2009-01-13T07:50:36.386011+00:00,1,5.40,-73.996506,40.747784,NaN,NaN,-73.940449,40.792385,Cash,15.4,0.0,0.0,0.0,15.4
2,VTS,2009-01-06T19:30:44.142187+00:00,2009-01-06T19:43:09.908429+00:00,5,1.67,-73.951340,40.752070,NaN,NaN,-73.944535,40.768108,CASH,6.1,1.0,0.0,0.0,7.1
3,VTS,2009-09-24T08:28:09.004451+00:00,2009-09-24T08:41:56.739991+00:00,1,4.13,-74.001117,40.729128,NaN,NaN,-73.958122,40.721812,CASH,12.9,0.5,0.0,0.0,13.4
4,VTS,2009-10-29T05:50:39.214629+00:00,2009-10-29T05:57:22.880174+00:00,1,2.03,-73.976600,40.756873,NaN,NaN,-73.979757,40.776075,CASH,7.7,0.5,0.0,0.0,8.2


In [7]:
df_csv.dtypes

vendor_id              object
pickup_datetime        object
dropoff_datetime       object
passenger_count         int64
trip_distance         float64
pickup_longitude      float64
pickup_latitude       float64
rate_code             float64
store_and_fwd_flag    float64
dropoff_longitude     float64
dropoff_latitude      float64
payment_type           object
fare_amount           float64
surcharge             float64
tip_amount            float64
tolls_amount          float64
total_amount          float64
dtype: object

In [10]:
df_csv['pickup_datetime'] = pd.to_datetime(df_csv['pickup_datetime'], format='mixed')
df_csv['dropoff_datetime'] = pd.to_datetime(df_csv['dropoff_datetime'], format='mixed')
df_csv.dtypes

vendor_id                          object
pickup_datetime       datetime64[ns, UTC]
dropoff_datetime      datetime64[ns, UTC]
passenger_count                     int64
trip_distance                     float64
pickup_longitude                  float64
pickup_latitude                   float64
rate_code                         float64
store_and_fwd_flag                float64
dropoff_longitude                 float64
dropoff_latitude                  float64
payment_type                       object
fare_amount                       float64
surcharge                         float64
tip_amount                        float64
tolls_amount                      float64
total_amount                      float64
dtype: object

In [11]:
df.head(2)

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,tip_amount,tolls_amount,total_amount
0,CMT,2009-04-21T18:51:11.767205+00:00,2009-04-21T18:57:09.433767+00:00,2,0.8,-74.004114,40.742950,NaN,NaN,-73.994712,40.747950,Cash,5.4,0.0,0.0,0.0,5.4
1,CMT,2009-01-13T07:40:07.639754+00:00,2009-01-13T07:50:36.386011+00:00,1,5.4,-73.996506,40.747784,NaN,NaN,-73.940449,40.792385,Cash,15.4,0.0,0.0,0.0,15.4


In [12]:
df_csv.to_parquet("../data/data-sample_data-nyctaxi-trips-2009-json_corrigido.parquet", compression='gzip')

In [13]:
from sqlalchemy import create_engine

In [14]:
import pymysql

In [15]:
user = "sprint"
password = "sprint"
host = "localhost"
port = "3306"
db = "sprint_ny_taxi"

In [16]:
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}")

In [17]:
engine.connect()

In [22]:
print(pd.io.sql.get_schema(df, name='nyctaxi-trips', con=engine))


CREATE TABLE nyctaxi-trips (
	vendor_id TEXT, 
	pickup_datetime TEXT, 
	dropoff_datetime TEXT, 
	passenger_count BIGINT, 
	trip_distance FLOAT(53), 
	pickup_longitude FLOAT(53), 
	pickup_latitude FLOAT(53), 
	rate_code FLOAT(53), 
	store_and_fwd_flag FLOAT(53), 
	dropoff_longitude FLOAT(53), 
	dropoff_latitude FLOAT(53), 
	payment_type TEXT, 
	fare_amount FLOAT(53), 
	surcharge FLOAT(53), 
	tip_amount FLOAT(53), 
	tolls_amount FLOAT(53), 
	total_amount FLOAT(53)
)




In [18]:
df_csv_iter = pd.read_csv('../data/data-sample_data-nyctaxi-trips-2009.csv', iterator=True, chunksize=100000)

In [19]:
df_csv = next(df_csv_iter)

In [20]:
len(df_csv)

100000

In [23]:
df_csv.head(n=0).to_sql(name='nyctaxi-trips', con=engine, if_exists='replace')

0

In [24]:
%time df_csv.to_sql(name='nyctaxi-trips', con=engine, if_exists='append')

CPU times: user 2.77 s, sys: 7.83 ms, total: 2.77 s
Wall time: 4.18 s


100000

In [25]:
from time import time

In [26]:
while True:
    t_start = time()
    
    df_csv = next(df_csv_iter)
    
    df_csv['pickup_datetime'] = pd.to_datetime(df_csv['pickup_datetime'], format='mixed')
    df_csv['dropoff_datetime'] = pd.to_datetime(df_csv['dropoff_datetime'], format='mixed')
    
    df_csv.to_sql(name='nyctaxi-trips', con=engine, if_exists='append')

    t_end = time()

    print("Another chunk inserted..., took %.3f second" % (t_end - t_start))


Another chunk inserted..., took 4.536 second
Another chunk inserted..., took 4.470 second
Another chunk inserted..., took 4.526 second
Another chunk inserted..., took 4.451 second
Another chunk inserted..., took 4.451 second
Another chunk inserted..., took 4.431 second
Another chunk inserted..., took 4.517 second
Another chunk inserted..., took 4.387 second
Another chunk inserted..., took 4.541 second


StopIteration: 